In [10]:
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext autotime
import mne
import numpy as np
from BrainPulse import (dataset,
                        vector_space,
                        distance_matrix,
                        recurrence_quantification_analysis,
                        features_space,
                        plot)
import pandas as pd
from tqdm import tqdm

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 2.39 ms (started: 2022-07-07 11:05:16 +02:00)


In [11]:
def normalize(tSignal):
    # copy the data if needed, omit and rename function argument if desired
    signal = np.copy(tSignal) # signal is in range [a;b]
    signal -= np.min(signal) # signal is in range to [0;b-a]
    signal /= np.max(signal) # signal is normalized to [0;1]
    signal -= 0.5 # signal is in range [-0.5;0.5]
    signal *=2 # signal is in range [-1;1]
    return signal

time: 292 µs (started: 2022-07-07 11:05:16 +02:00)


In [12]:
number_of_subjects = [0,50]
t_start = 0
t_end = 30
fir_filter = [2.0, 50.0]

min_vert_line_len_ = 2
win_len = 240
n_fft = 512

percentile_var = 4

results_parameters_tdemb = []
results_parameters_stft = []

time: 4.34 ms (started: 2022-07-07 11:05:16 +02:00)


In [ ]:

for n in tqdm(range(number_of_subjects[0]+1,number_of_subjects[1])):
    print(n)
    Subject_id = 'S'+ str(f'{n:03}')
    run, raw = dataset.eegbci_data(tmin=t_start, tmax=t_end, subject=n, filter_range=fir_filter)

    for task, task_io in zip(run, ['open_eyes','closed_eyes']):
        for eid in tqdm(range(64)):
        # for eid, electrode in enumerate(task):
        #     signal = electrode * 1e6
            signal = task[eid] * 1e6

            norm_signal = normalize(signal)

            matrix_tdemb = distance_matrix.EuclideanPyRQA_RP(norm_signal, embedding=5, timedelay=9)
            nbr_tdemb = np.percentile(matrix_tdemb, percentile_var)
            matrix_binary_tdemb = distance_matrix.set_epsilon(matrix_tdemb, nbr_tdemb)

            result_rqa_tdemb = recurrence_quantification_analysis.get_results(matrix_binary_tdemb,2,min_vert_line_len_,2)
            results_parameters_tdemb.append([Subject_id, task_io, run.ch_names[eid]] + result_rqa_tdemb)
            # plt.imshow(matrix_binary_tdemb, cmap='cividis', origin='lower')

            cut_freq = 60
            stft = vector_space.compute_stft(norm_signal,
                                             n_fft=n_fft, win_len=win_len,
                                             s_rate=run.info['sfreq'],
                                             cut_freq=cut_freq)

            matrix_stft = distance_matrix.EuclideanPyRQA_RP_stft(stft)
            nbr_stft = np.percentile(matrix_stft, percentile_var)
            matrix_binary_stft = distance_matrix.set_epsilon(matrix_stft, nbr_stft)

            result_rqa_stft = recurrence_quantification_analysis.get_results(matrix_binary_stft,2,min_vert_line_len_,2)
            results_parameters_stft.append([Subject_id, task_io, run.ch_names[eid]] + result_rqa_stft)


In [14]:
df_tdemb = pd.DataFrame(results_parameters_tdemb,
                        columns=['Subject', 'Task', 'Electrode',
                                 'RR', 'DET', 'L', 'Lmax', 'DIV',
                                 'Lentr', 'DET_RR', 'LAM', 'V',
                                 'Vmax', 'Ventr', 'LAM_DET', 'W',
                                 'Wmax', 'Wentr', 'TT'])

time: 16.2 ms (started: 2022-07-07 13:00:34 +02:00)


In [15]:
df_stft = pd.DataFrame(results_parameters_stft,
                       columns=['Subject', 'Task', 'Electrode',
                                'RR', 'DET', 'L', 'Lmax', 'DIV',
                                'Lentr', 'DET_RR', 'LAM', 'V',
                                'Vmax', 'Ventr', 'LAM_DET', 'W',
                                'Wmax', 'Wentr', 'TT'])

time: 16.6 ms (started: 2022-07-07 13:00:34 +02:00)


In [16]:
df_tdemb.to_csv('df_tdemb_4th_'+str(number_of_subjects[1])+'.csv')

time: 65.8 ms (started: 2022-07-07 13:00:34 +02:00)


In [17]:
df_stft.to_csv('df_stft_4th_'+str(number_of_subjects[1])+'.csv')

time: 70.1 ms (started: 2022-07-07 13:00:35 +02:00)


In [18]:
df_stft.head()

,Subject,Task,Electrode,RR,DET,L,Lmax,DIV,Lentr,DET_RR,LAM,V,Vmax,Ventr,LAM_DET,W,Wmax,Wentr,TT
0,S001,open_eyes,FC5,0.04,0.999961,56.770697,4801,0.000208,4.403802,24.999078,0.999984,65.853991,361,5.063298,1.000023,1181.702450,4766,7.712069,66.784607
1,S001,open_eyes,FC3,0.04,0.999937,53.869657,4801,0.000208,4.322867,24.998424,0.999984,61.753682,242,4.975233,1.000047,1125.868833,4761,7.700242,62.692730
2,S001,open_eyes,FC1,0.04,0.999956,56.021859,4801,0.000208,4.357343,24.998914,0.999984,64.536314,248,5.024931,1.000027,1163.871399,4763,7.746830,65.469704
3,S001,open_eyes,FCz,0.04,0.999937,52.597340,4801,0.000208,4.278699,24.998424,0.999985,62.359137,258,4.971709,1.000048,1134.465895,4761,7.700304,63.301113
4,S001,open_eyes,FC2,0.04,0.999946,62.834954,4801,0.000208,4.424272,24.998642,0.999986,70.787030,294,5.086988,1.000040,1247.215843,4758,7.768188,71.717476


time: 5.89 ms (started: 2022-07-07 13:00:35 +02:00)
